## 1 Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import base64
import sys
import os
from twocaptcha import TwoCaptcha
from selenium.common.exceptions import TimeoutException

## 2 Declarando Parâmetros

In [2]:
# Defina o caminho correto para o ChromeDriver (não o Chrome .lnk)
chrome_driver_path = r'C:\Users\fluca\Desktop\dev\chromedriver-win32\chromedriver.exe'

In [3]:
# Inicializando o serviço do ChromeDriver
service = Service(executable_path=chrome_driver_path)

In [12]:
# Inicializando o navegador Chrome com o serviço
driver = webdriver.Chrome(service=service)

In [ ]:
# Navega para o site
driver.get('http://www.sga.pr.gov.br/sga-iap/consultarProcessoLicenciamento.do?action=iniciar#')

In [13]:
# Chave da API 2Captcha
api_key = 'eb6dc8fd6566313857572a8dbd744b49'

In [14]:
solver = TwoCaptcha(api_key)

## 3 Funções

In [4]:
# Caminho da pasta onde os CAPTCHAs serão salvos
captcha_dir = r"C:\Users\fluca\Desktop\Data Science\Projetos\webscraping_inst_agua_e_terra_pr\downloads_captchas"

# Configurações da API do 2Captcha
api_key = 'eb6dc8fd6566313857572a8dbd744b49'
solver = TwoCaptcha(api_key)

In [5]:
# Função para inicializar o navegador e acessar o site
def inicializar_navegador(url):
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    print(f"Carregou o site: {url}")
    return driver

In [6]:
# Função para clicar no botão "Pesquisa Avançada"
def clicar_pesquisa_avancada(driver):
    try:
        wait = WebDriverWait(driver, 3)
        pesquisa_avancada_button = wait.until(EC.element_to_be_clickable((By.ID, 'ckPesquisaAvancada-inputEl')))
        pesquisa_avancada_button.click()
        print("Clicou no botão 'Pesquisa Avançada'.")
        time.sleep(3)
    except Exception as e:
        print(f"Erro ao clicar em 'Pesquisa Avançada': {str(e)}")

In [7]:
# Função para preencher o campo "Grupo"
def preencher_grupo(driver, grupo):
    try:
        wait = WebDriverWait(driver, 3)
        grupo_field = wait.until(EC.presence_of_element_located((By.ID, 'grupoCombo-inputEl')))
        grupo_field.click()
        grupo_field.clear()
        grupo_field.send_keys(grupo)
        time.sleep(3)
        grupo_field.send_keys(Keys.RETURN)
        print(f"Preencheu o campo 'Grupo' com '{grupo}'.")
        time.sleep(3)
    except Exception as e:
        print(f"Erro ao preencher o campo 'Grupo': {str(e)}")

In [8]:
# Função para preencher o campo "Modalidade"
def preencher_modalidade(driver, modalidade):
    try:
        wait = WebDriverWait(driver, 3)
        modalidade_field = wait.until(EC.presence_of_element_located((By.ID, 'modalidadeCombo-inputEl')))
        modalidade_field.click()
        modalidade_field.clear()
        modalidade_field.send_keys(modalidade)
        time.sleep(3)
        modalidade_field.send_keys(Keys.RETURN)
        print(f"Preencheu o campo 'Modalidade' com '{modalidade}'.")
        time.sleep(3)
    except Exception as e:
        print(f"Erro ao preencher o campo 'Modalidade': {str(e)}")

In [9]:
# Função para clicar no botão "Pesquisar"
def clicar_pesquisar(driver):
    try:
        wait = WebDriverWait(driver, 3)
        botao_pesquisar = wait.until(EC.element_to_be_clickable((By.ID, 'botaoPesquisar_consultarProcessoLicenciamentoGrid-btnInnerEl')))
        botao_pesquisar.click()
        print("Clicou no botão 'Pesquisar'.")
        time.sleep(3)
    except Exception as e:
        print(f"Erro ao clicar no botão 'Pesquisar': {str(e)}")

In [10]:
# Função para clicar na lupa correspondente
def clicar_lupa(driver, index):
    try:
        wait = WebDriverWait(driver, 3)
        lupa_icon = wait.until(EC.presence_of_element_located((By.XPATH, f"(//img[@src='images/icon_exibir.png'])[{index}]")))
        lupa_icon.click()
        time.sleep(3)
        print(f"Clicou no ícone da lupa {index}.")
    except TimeoutException:
        print(f"Erro: Lupa {index} não encontrada.")
        return False
    return True

In [11]:
# Função para clicar no botão "Visualizar Documento de Licença"
def clicar_visualizar_documento(driver):
    try:
        wait = WebDriverWait(driver, 3)
        visualizar_documento_btn = wait.until(EC.element_to_be_clickable((By.ID, 'btnPesquisarGeradorResiduo-btnInnerEl')))
        visualizar_documento_btn.click()
        time.sleep(3) 
        print("Clicou em 'Visualizar Documento de Licença'.")
    except Exception as e:
        print(f"Erro ao clicar em 'Visualizar Documento de Licença': {str(e)}")

In [12]:
# Função para baixar a imagem do CAPTCHA
def baixar_captcha(driver, index):
    try:
        captcha_element = driver.find_element(By.ID, "gera_captcha")
        captcha_src = captcha_element.get_attribute("src")
        
        if captcha_src.startswith('data:image/jpeg;base64,'):
            captcha_base64 = captcha_src.replace('data:image/jpeg;base64,', '')
        elif captcha_src.startswith('data:image/png;base64,'):
            captcha_base64 = captcha_src.replace('data:image/png;base64,', '')
        else:
            raise ValueError("Formato de imagem não suportado")
        
        captcha_path = os.path.join(captcha_dir, f"captcha{index}.jpg")
        captcha_data = base64.b64decode(captcha_base64)
        with open(captcha_path, "wb") as f:
            f.write(captcha_data)

        print(f"Imagem do captcha {index} salva com sucesso.")
        return captcha_path

    except Exception as e:
        print(f"Erro ao baixar o CAPTCHA {index}: {str(e)}")
        return None

In [13]:
# Função para resolver o CAPTCHA usando a API 2Captcha
def resolver_captcha(captcha_path):
    try:
        result = solver.normal(captcha_path)
        captcha_resolvido = result['code']
        print(f"Captcha resolvido: {captcha_resolvido}")
        return captcha_resolvido
    except Exception as e:
        print(f"Erro ao resolver o CAPTCHA: {str(e)}")
        return None

In [14]:
# Função para inserir o CAPTCHA e clicar no botão "Continuar"
def inserir_captcha_e_continuar(driver, captcha_resolvido):
    try:
        wait = WebDriverWait(driver, 3)
        captcha_input = wait.until(EC.presence_of_element_located((By.ID, 'captchaDigitada-inputEl')))
        captcha_input.clear()
        captcha_input.send_keys(captcha_resolvido)
        print("Captcha inserido no campo de texto.")
        time.sleep(3)
        
        botao_continuar = wait.until(EC.element_to_be_clickable((By.ID, 'button-1057-btnInnerEl')))
        botao_continuar.click()
        print("Clicou no botão 'Continuar'.")
        
        time.sleep(5)  # Aguarda para carregar a próxima página

        # Verifica se a mensagem de erro de CAPTCHA aparece
        mensagem_erro_captcha = driver.find_elements(By.XPATH, "//div[@id='fileInfo-inputEl' and contains(text(), 'Ops..CAPTCHA  inválido')]")
        if mensagem_erro_captcha:
            print("CAPTCHA incorreto, tentando novamente...")
            return False  # Indica que o CAPTCHA falhou
        else:
            print("CAPTCHA resolvido com sucesso.")
            return True  # CAPTCHA foi resolvido corretamente

    except Exception as e:
        print(f"Erro ao inserir o CAPTCHA e continuar: {str(e)}")
        return False  # Em caso de erro, tenta de novo


In [15]:
# Função para clicar no botão "Pesquisar"
def clicar_pesquisar(driver):
    try:
        wait = WebDriverWait(driver, 3)
        botao_pesquisar = wait.until(EC.element_to_be_clickable((By.ID, 'botaoPesquisar_consultarProcessoLicenciamentoGrid-btnInnerEl')))
        botao_pesquisar.click()
        print("Clicou no botão 'Pesquisar' novamente.")
        time.sleep(3)  # Tempo para garantir que a pesquisa seja concluída e as lupas apareçam
    except Exception as e:
        print(f"Erro ao clicar no botão 'Pesquisar': {str(e)}")

In [16]:
# Função principal para processar as lupas e resolver os CAPTCHAs
def processar_lupas(driver, num_lupas):
    for i in range(1, num_lupas + 1):
        try:
            if not clicar_lupa(driver, i):
                continue
            
            time.sleep(3)  # Espera pelo carregamento da nova página

            # Clica no botão "Visualizar Documento de Licença"
            clicar_visualizar_documento(driver)
            
            captcha_resolvido = False
            while not captcha_resolvido:
                # Baixa o CAPTCHA
                captcha_path = baixar_captcha(driver, i)
                if not captcha_path:
                    continue
                
                # Resolve o CAPTCHA
                captcha_resolvido_str = resolver_captcha(captcha_path)
                if not captcha_resolvido_str:
                    continue

                time.sleep(3)

                # Insere o CAPTCHA e tenta continuar
                captcha_resolvido = inserir_captcha_e_continuar(driver, captcha_resolvido_str)
                if not captcha_resolvido:
                    print("Tentando resolver o CAPTCHA novamente...")

            time.sleep(3)

            # Volta à página principal
            driver.back()
            print(f"Retornou à página principal após processar a lupa {i}.")
            time.sleep(3)

            # Clica no botão "Pesquisar" para recarregar as lupas
            clicar_pesquisar(driver)

        except Exception as e:
            print(f"Erro ao processar a lupa {i}: {str(e)}")


In [17]:
# Função principal para iniciar o processo completo
def processar_pesquisa_completa(url, grupo, modalidade, num_lupas):
    driver = inicializar_navegador(url)
    
    try:
        clicar_pesquisa_avancada(driver)
        preencher_grupo(driver, grupo)
        preencher_modalidade(driver, modalidade)
        clicar_pesquisar(driver)
        processar_lupas(driver, num_lupas)
    
    except Exception as e:
        print(f"Erro durante o processamento: {str(e)}")
    
    finally:
        # Fecha o navegador ao final do processo
        driver.quit()
        print("Navegador fechado.")

## 4 Execução

In [18]:
# Executa a função principal para processar as lupas com os parâmetros necessários
url = 'http://www.sga.pr.gov.br/sga-iap/consultarProcessoLicenciamento.do?action=iniciar#'
grupo = 'Agropecuária'
modalidade = 'LO - Licença de operação'

In [20]:
processar_pesquisa_completa(url, grupo, modalidade, 10)

Carregou o site: http://www.sga.pr.gov.br/sga-iap/consultarProcessoLicenciamento.do?action=iniciar#
Clicou no botão 'Pesquisa Avançada'.
Preencheu o campo 'Grupo' com 'Agropecuária'.
Preencheu o campo 'Modalidade' com 'LO - Licença de operação'.
Clicou no botão 'Pesquisar' novamente.
Clicou no ícone da lupa 1.
Clicou em 'Visualizar Documento de Licença'.
Imagem do captcha 1 salva com sucesso.
Captcha resolvido: 4dpou
Captcha inserido no campo de texto.
Clicou no botão 'Continuar'.
CAPTCHA resolvido com sucesso.
Retornou à página principal após processar a lupa 1.
Clicou no botão 'Pesquisar' novamente.
Clicou no ícone da lupa 2.
Clicou em 'Visualizar Documento de Licença'.
Imagem do captcha 2 salva com sucesso.
Captcha resolvido: 1xdyo
Navegador fechado.


KeyboardInterrupt: 